In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
#import matplotlib.pyplot as plt

In [2]:
df =  pd.read_csv('./2017_07_07/TADPOLE_D1_D2.csv')

C:\Users\antoi\AppData\Local\Temp\ipykernel_13656\3674165071.py:1: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,72

In [30]:
df.head(0)
test = df[1:]
test.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 1 to 2999
Data columns (total 1907 columns):
 #     Column                                                            Dtype  
---    ------                                                            -----  
 0     RID                                                               int64  
 1     PTID                                                              object 
 2     VISCODE                                                           object 
 3     SITE                                                              int64  
 4     D1                                                                int64  
 5     D2                                                                int64  
 6     COLPROT                                                           object 
 7     ORIGPROT                                                          object 
 8     EXAMDATE                                                          object 


In [4]:
df.shape

(12741, 1907)

In [5]:
df_sample = df.sample(frac=0.1, random_state=1)
pd.set_option('display.max_columns', None)
name_columns=df.head(0)
#EXAMDATE	DX_bl	DXCHANGE	AGE	PTGENDER	PTEDUCAT	PTETHCAT	PTRACCAT	PTMARRY	


In [6]:
print(df['DXCHANGE'].head(1))
'1=Stable:NL to NL, 2=Stable:MCI to MCI, 3=Stable:AD to AD, ' 
'4=Conv:NL to MCI, 5=Conv:MCI to AD, 6=Conv:NL to AD, ' 
'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

0    1.0
Name: DXCHANGE, dtype: float64


'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

In [7]:
from sklearn.neighbors import KNeighborsClassifier
df=df[:3000]
gender=df['PTGENDER']
age=df['AGE'].astype(float)
education=df['PTEDUCAT']
gene_APOE4=df['APOE4'].astype(float)
diagnostic=df['DXCHANGE'].astype(float)

In [21]:

data = df[['PTGENDER','AGE','PTEDUCAT','APOE4','DXCHANGE','D1','D2']].copy()
data.dropna(axis=0,inplace= True)
data['PTGENDER']= data['PTGENDER'].replace(['Male','Female'],[1,0])
data['APOE4']= data['APOE4'].astype(float)
data['AGE']= data['AGE'].astype(float)
#data = data[:1000]
data.shape

(3000, 7)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PTGENDER  3000 non-null   int64  
 1   AGE       3000 non-null   float64
 2   PTEDUCAT  3000 non-null   int64  
 3   APOE4     3000 non-null   float64
 4   DXCHANGE  3000 non-null   float64
 5   D1        3000 non-null   int64  
 6   D2        3000 non-null   int64  
dtypes: float64(3), int64(4)
memory usage: 164.2 KB


In [22]:

print(type(age))
gender= gender.replace(['Male','Female'],[1,0])

<class 'pandas.core.series.Series'>


In [35]:
# Algorithme des k plus proches voisins
k=3 # best beetween 1 and 10
model = KNeighborsClassifier()
X_train = data[:1]
X_test = data[:1]
X_test.head()

for i in range(len(data)):
    if data['D1'][i] == 1:
        X_train.add(data[i])
    elif data['D2'][i] == 1:
        X_test.add(data[i])
y_train = X_train['DXCHANGE']
y_test = X_test['DXCHANGE']
X_train = data.drop('DXCHANGE',axis=1)
X_test = data.drop('DXCHANGE',axis=1)

model.fit(X_train,y_train)
print(model.score(X_train,y_train))
    

KeyError: 0

In [11]:
def sick(model,PTGENDER=1,AGE=10,PTEDUCAT=4,APOE4=0):
    x= np.array([PTGENDER,AGE,PTEDUCAT,APOE4]).reshape(1,4)
    print(model.predict(x))
    print(model.predict_proba(x))

In [12]:
sick(model)

[2.]
[[0.  0.8 0.2 0.  0.  0.  0. ]]


C:\Users\antoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\antoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [13]:
param_grid = {'n_neighbors': np.arange(1, 25),'metric': ['euclidean', 'manhattan']}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)

grid.fit(X, y)

C:\Users\antoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [17]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

{'metric': 'manhattan', 'n_neighbors': 21}
0.43800000000000006
KNeighborsClassifier(metric='manhattan', n_neighbors=21)
